# Mapping Change Detection Results 

This is an intro notebook on using geospatial data packages like folium and H3 to aggregate and visualize polygon  from the Planet change detection analytic feed using hierarchical indices. H3 is Uber's spatial indexing system using a hexagonal grid system optimized for visualizing geospatial data. You can read more here (https://eng.uber.com/h3/). Folium provides an intuitive python library for creating layered maps on leaflet. You can learn more here (https://www.coursera.org/lecture/python-for-data-visualization/introduction-to-folium-CpjW0). You can learn more about Planet's analytic feeds here (https://www.planet.com/products/analytics/) and request access to the building change detection feed here (https://learn.planet.com/explore-product-announcement.html)

In [1]:
import requests
import os
import json
import pandas as pd
from h3 import h3
from pprint import pprint
from IPython.display import IFrame
import folium
import webbrowser
from folium import Map
import geopandas as gpd
from dateutil.parser import parse

## Helper Functions

Defining some helper functions to get define h3 indices from rows with geometriesparse subscription results

In [2]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['lat'], row['lng'], H3_LEVEL)

def get_next_link(results_json):
    """Given a response json from one page of subscription results, get the url for the next page of results."""
    for link in results_json['links']:
        if link['rel'] == 'next':
            return link['href']
    return None

## Data access and preparation

Paginating through a change detection subscriptions, accessing results and capturing those results into a geopandas dataframe

In [9]:
LIVE_BASE_URL = 'https://api.planet.com/analytics/'
# if your Planet API Key is not set as an environment variable, you can paste it below
PL_API_KEY = os.environ['PL_API_KEY']
# construct auth tuple for use in the requests library
BASIC_AUTH = (PL_API_KEY, '')
EXISTING_GEOJSON = True
H3_LEVEL = 4

'''
bbox = left,bottom,right,top
bbox = min Longitude , min Latitude , max Longitude , max Latitude
'''
# Noting Subscription IDs and associated bounding boxes we need to filter API search
SUB_ID_CA = '0163608e-0afb-477e-a500-3a87819351a1'
BBOX_LONGBEACH = '-118.24896793,33.710663987,-118.06315999,33.8854059'
BBOX_RIVERSIDE = '-117.79347778,33.62802278,-116.90089412,34.2035886'

SUBSCRIPTION_ID = SUB_ID_CA
BBOX = BBOX_RIVERSIDE

if EXISTING_GEOJSON:
    gdf = gpd.read_file('./data/collection_0163608e-0afb-477e-a500-3a87819351a1.geojson')
else:
    # results_url = LIVE_BASE_URL + 'collections/' + SUBSCRIPTION_ID + '/items?bbox=' + BBOX
    results_url = LIVE_BASE_URL + 'collections/' + SUBSCRIPTION_ID + '/items'
    # Get subscription results collection
    resp = requests.get(results_url, auth=BASIC_AUTH)
    #checking access
    if resp.status_code == 200:
        print('Yay, you can access analytic feed results!')
        subscription_results = resp.json()
    else:
        print('Something is wrong:', resp.content)

    latest_feature = subscription_results['features'][0]
    creation_datestring = latest_feature['created']
    print('latest feature creation date:', creation_datestring)

    feature_collection = {'type': 'FeatureCollection', 'features': []}
    next_link = results_url

    bufffer_count =0

    while next_link:
        bufffer_count += 1
        print('result collection : ', bufffer_count)
        results = requests.get(next_link, auth=BASIC_AUTH).json()
        next_features = results['features']
        if next_features:
            latest_feature_creation = parse(next_features[0]['created']).date()
            earliest_feature_creation = parse(next_features[-1]['created']).date()
            print('Fetched {} features fetched ({}, {})'.format(
                len(next_features), earliest_feature_creation, latest_feature_creation))
            feature_collection['features'].extend(next_features)
            next_link = get_next_link(results)
        else:
            next_link = None

    print('Total features: {}'.format(len(feature_collection['features'])))

    filename = 'collection_{}.geojson'.format(SUBSCRIPTION_ID)
    with open(filename, 'w') as file:
        json.dump(feature_collection, file)

    gdf = gpd.read_file(filename)

## Centroid detection and indexing

In [10]:
gdf['lat'] = gdf.centroid.map(lambda p: p.y)
gdf['lng'] = gdf.centroid.map(lambda p: p.x)
gdf['h3'] = gdf.apply(lat_lng_to_h3, axis=1)

# What does the data from Planet look like?

In [8]:
print(gdf.shape)
print(gdf.iloc[0])

(21743, 12)
id                                   67093241-0c50-4f73-aacf-c75f4566a3d5
label                                                            building
object_area_m2                                                    773.879
observed                                              2019-06-01T00:00:00
source_last_acquired                                  2019-07-01T00:00:00
source_mosaic                                                        None
source_mosaic_name                          global_monthly_2019_06_mosaic
source_quad_id                                                   340-1284
geometry                POLYGON ((-120.177512152137 41.52711793718, -1...
lat                                                                41.527
lng                                                              -120.177
h3                                                        842812bffffffff
Name: 0, dtype: object


# Hierarchical Clustering

Aggregate change polygons by h3 cells and derive a count of detected change assets. We define a few functions to hierarchical clustering of the results and create a color map based on the counts within in each hexagonal cell

In [5]:
# Clustering with H3 is easy. 
# Just iterate through all points and aggregate the hit counts for each key. 
# Here we don't care about noise, all points are good.
def get_relevant_clusters(gdf):
    #Empty dict
    h3_clusters = dict()
    for index, row in gdf.iterrows():
        key = row['h3']
        if key in h3_clusters:
            h3_clusters[key]["count"] += 1
        else:
            h3_clusters[key] = {"count": 1,
                                "geom": h3.h3_to_geo_boundary(h3_address=key)}

    # relevant cluseters
    relevant_clusters = { key:value for (key,value) in h3_clusters.items() if value['count'] >= 1}

    return relevant_clusters

def create_map(clusters):
    # The create_map function creates and populates a folium map from the given 
    # clusters dictionary. Each value of the dictionary is also a dictionary with 
    # two recognizable keys: geom and count. The value associated with the geom key 
    # is the list of locations of the hexagon's vertices. The value associated with
    # the count key contains the number of accidents counted within this shape.
    # Create the map object
    map1 = Map(tiles="cartodbpositron", 
          attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>')

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        # points = [p[::-1] for p in points]
        tooltip = "{0} assets".format(cluster['count'])
        polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color='#ff0000', 
                                               fill_color='#ff0000', 
                                               fill_opacity=0.4, weight=3, opacity=0.4)
        polygon.add_to(map1)

    # Determine the map bounding box
    max_lat = gdf.lat.max()
    min_lat = gdf.lat.min()
    max_lon = gdf.lng.max()
    min_lon = gdf.lng.min()
    
    # Fit the map to the bounds
    map1.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map1

def create_gradient_map(clusters):
    # Create the map object
    map1 = Map(tiles="cartodbpositron", 
          attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>')


    max_count = max([item['count'] for item in clusters.values()])

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        if cluster['count'] > 20: #some minimum threshold to filter out low volume build up
            color = '#' + hex(int((0.25+(cluster['count']/(1.34*max_count)))*255))[2:].zfill(2) + '0000'
            # color = '#' + hex(int((cluster['count']/max_count)*255))[2:].zfill(2) + '0000'
            # points = [p[::-1] for p in points]
            tooltip = "{0} assets".format(cluster['count'])
            polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color=color, 
                                               fill_color=color, 
                                               fill_opacity=0.8, weight=3, opacity=0.0)
            polygon.add_to(map1)

    # Determine the map bounding box
    max_lat = gdf.lat.max()
    min_lat = gdf.lat.min()
    max_lon = gdf.lng.max()
    min_lon = gdf.lng.min()
    
    # Fit the map to the bounds
    map1.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map1

def show_map(map1, file_name):
    # The show_map function saves the HTML generated by the map into a file and then opens a new browser tab with its contents
    map1.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)


# Do the Clustering

Creating additional columns for centroids of the change detection results and H3 index, clustering loading them to a map

In [6]:
relevant_clusters = get_relevant_clusters(gdf)
h3_map = create_gradient_map(relevant_clusters)

# Visualizing a Change Heatmap

We store the results with the subscription ID and zoom level and load it my in an IFrame

In [7]:
h3_map.save("./data/{0}_change_map_h3_{1}.html".format(SUBSCRIPTION_ID, H3_LEVEL))
IFrame(src="./data/{0}_change_map_h3_{1}.html".format(SUBSCRIPTION_ID, H3_LEVEL), width=700, height=600)